# Fed vs. fasted: comparing RSNs in giant Mappers

Let's recreate Figure 4 in Poldrack et al. 2015, Nature Communications, using Mapper!

#### Load data

In [1]:
from utils import *

majorRSNs = ['Cingulo_opercular','Frontoparietal_1','Somatomotor','Visual_2','DMN']
 
df_behavior = pd.read_table('data/base/behavior/trackingdata_goodscans.txt')

sessions = load_scrubbed(n_sessions=-1, merge=False)

roi2rsn = np.array(sessions[0].atlas['data']['network'])
roi2rsn_clean = roi2rsn[np.isin(roi2rsn,majorRSNs)]

/Users/rafi/miniconda/envs/dyno/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
INFO:load_data:load_scrubbed(**{'n_sessions': -1, 'merge': False})
INFO:load_data:Loading data...
INFO:root:  [+] session: 1, path: data/base/combined_data_scrubbed/sub014.txt
INFO:root:  [+] session: 2, path: data/base/combined_data_scrubbed/sub015.txt
INFO:root:  [+] session: 3, path: data/base/combined_data_scrubbed/sub016.txt
INFO:root:  [+] session: 4, path: data/base/combined_data_scrubbed/sub017.txt
INFO:root:  [+] session: 5, path: data/base/combined_data_scrubbed/sub018.txt
INFO:root:  [+] session: 6, path: data/base/combined_data_scrubbed/sub019.txt
INFO:root:  [+] session: 7, path: data/base/combined_data_scrubbed/sub020.txt
INFO:root:  [+] session: 8, path: data/base/combined_data_scrubbed/sub021.txt
INFO:root:  [+] session: 9, path: data/base/combined_data_scrubbed/sub022.txt


#### Clean data

Load temporal masks and remove TRs with excessive motion. Additionally, remove parcels that do not belong to major RSNs.

In [2]:
def clean_data(datamat, df_rsn_act, subject_index):
    # Datamat must be TRs x ROIs
    newdata = np.array(datamat)
    # Get subcode
    subcode = df_behavior.loc[subject_index,'subcode']
    # Load temporal mask
    tmask = np.loadtxt('data/base/rsfmri/tmasks/' + subcode + '.txt').astype(bool)
    # Clean data
    newdata = newdata[tmask,:]
    newdata = newdata[:,np.isin(roi2rsn,majorRSNs)]
    # Clean the one hot encoded pandas dataframe df_rsn_act
    newdf = df_rsn_act.iloc[tmask,np.isin(df_rsn_bin.columns,majorRSNs)]

    return newdata,newdf    

#### Run Mapper in temporal space - create one super graph for both fed and fasted

In [4]:
fed_frames = []
fast_frames = []
X_fed = []
X_fast = []
for sess_num,session in enumerate(sessions):
    # Get RSN activations
    df_rsn_bin = get_RSN_act(session.X, session.atlas)
    # Clean data
    X,df = clean_data(session.X,df_rsn_bin,sess_num)
    # Zscore data
    Xz = scipy.stats.zscore(X,axis=0)
    # Append to big list - tuesday is fasted, thursday is fed
    if int(session.meta['day_of_week'][0]) == 4:
        X_fed.append(Xz)
        fed_frames.append(df)
    elif int(session.meta['day_of_week'][0]) == 2:
        X_fast.append(Xz)
        fast_frames.append(df)

df_rsn_fed = pd.concat(fed_frames)
df_rsn_fast = pd.concat(fast_frames)
X_fed = np.concatenate(X_fed,axis=0)
X_fast = np.concatenate(X_fast,axis=0)
print(X_fed.shape)
print(X_fast.shape)
print(df_rsn_fed.shape)
print(df_rsn_fast.shape)

(15576, 437)
(20262, 437)
(15576, 5)
(20262, 5)


In [ ]:
# Sweep through resolution and gain
resolutions = [10,30,50,70]
gains = [2.,3.,4.,5.]
itera = 1
for res in resolutions:
    for gain in gains:
        overlap = (gain - 1) / gain
        # Run Mapper/DyNeuSR
        print('Running mapper %d on fed data...' % itera)
        results_fed = run_mapper(X_fed,df_rsn_fed,verbose=1,cover=km.Cover(res,overlap))
        # Visualize DyNeuGraph
        results_fed['dG'].visualize(path_html='results/fed_super_R%d_G%d.html' % (res,int(gain)), show=False, reset=True, static=False, PORT=8800)
        itera = itera + 1

Running mapper 1 on fed data...
KeplerMapper()
Mapping on data shaped (15576, 437) using lens shaped (15576, 2)

Creating 100 hypercubes.

Created 306 edges and 91 nodes in 0:00:16.753930.


label,group,value,row_count
Cingulo_opercular,0,2882,15575
DMN,1,3136,15575
Frontoparietal_1,2,2921,15575
Somatomotor,3,2733,15575
Visual_2,4,3044,15575


[Force Graph] http://localhost:8800/results/fed_super_R10_G2.html
Running mapper 2 on fed data...
KeplerMapper()
Mapping on data shaped (15576, 437) using lens shaped (15576, 2)

Creating 100 hypercubes.

Created 318 edges and 94 nodes in 0:00:23.542975.


label,group,value,row_count
Cingulo_opercular,0,2882,15575
DMN,1,3136,15575
Frontoparietal_1,2,2921,15575
Somatomotor,3,2733,15575
Visual_2,4,3044,15575


[Force Graph] http://localhost:8800/results/fed_super_R10_G3.html
Running mapper 3 on fed data...
KeplerMapper()
Mapping on data shaped (15576, 437) using lens shaped (15576, 2)

Creating 100 hypercubes.

Created 322 edges and 95 nodes in 0:00:27.405183.


label,group,value,row_count
Cingulo_opercular,0,2882,15575
DMN,1,3136,15575
Frontoparietal_1,2,2921,15575
Somatomotor,3,2733,15575
Visual_2,4,3044,15575


[Force Graph] http://localhost:8800/results/fed_super_R10_G4.html
Running mapper 4 on fed data...
KeplerMapper()
Mapping on data shaped (15576, 437) using lens shaped (15576, 2)

Creating 100 hypercubes.

Created 322 edges and 95 nodes in 0:00:30.993773.


label,group,value,row_count
Cingulo_opercular,0,2882,15575
DMN,1,3136,15575
Frontoparietal_1,2,2921,15575
Somatomotor,3,2733,15575
Visual_2,4,3044,15575


In [ ]:
# Sweep through resolution and gain
resolutions = [10,30,50,70]
gains = [2.,3.,4.,5.]
itera = 1
for res in resolutions:
    for gain in gains:
        overlap = (gain - 1) / gain
        # Run Mapper/DyNeuSR
        print('Running mapper %d on fast data...' % itera)
        results_fast = run_mapper(X_fast,df_rsn_fast,verbose=1,cover=km.Cover(res,overlap))
        # Visualize DyNeuGraph
        results_fast['dG'].visualize(path_html='results/fast_super_R%d_G%d.html' % (res,int(gain)), show=False, reset=True, static=False, PORT=8800)
        itera = itera + 1

#### Run Mapper in anatomical space

In [3]:
df_roi_rsn = pd.DataFrame(0,index=np.arange(roi2rsn_clean.shape[0]),columns=majorRSNs)
for ind,roi in enumerate(roi2rsn_clean):
    df_roi_rsn.loc[ind,roi] = 1

In [4]:
fed_frames = []
fast_frames = []
X_fed = []
X_fast = []
for sess_num,session in enumerate(sessions):
    # Get RSN activations
    df_rsn_bin = get_RSN_act(session.X, session.atlas)
    # Clean data
    X,df = clean_data(session.X,df_rsn_bin,sess_num)
    # Zscore data
    Xz = scipy.stats.zscore(X,axis=0)
    # Append to big list - tuesday is fasted, thursday is fed
    if int(session.meta['day_of_week'][0]) == 4:
        X_fed.append(Xz)
    elif int(session.meta['day_of_week'][0]) == 2:
        X_fast.append(Xz)

X_fed = np.concatenate(X_fed,axis=0)
X_fast = np.concatenate(X_fast,axis=0)
print(X_fed.shape)
print(X_fast.shape)

(15576, 437)
(20262, 437)


In [7]:
res = 25
gain = 5
overlap = (gain - 1) / gain
# Run Mapper/DyNeuSR
results_fed = run_mapper(X_fed.T,df_roi_rsn,verbose=1,cover=km.Cover(res,overlap))
# Visualize DyNeuGraph
results_fed['dG'].visualize(path_html='results/fed_super_spatial.html', show=False, reset=True, static=False, PORT=8800)

KeplerMapper()
Mapping on data shaped (437, 15576) using lens shaped (437, 2)

Creating 625 hypercubes.

Created 317 edges and 149 nodes in 0:00:00.854166.


label,group,value,row_count
Cingulo_opercular,0,115,436
Frontoparietal_1,1,112,436
Somatomotor,2,90,436
Visual_2,3,67,436
DMN,4,52,436


[Force Graph] http://localhost:8800/results/fed_super_spatial.html


DyNeuGraph()